<a href="https://colab.research.google.com/github/Brand-Sentiment-Tracking/python-package/blob/main/johnsnow/CLASS_FOR_SENTIMENT_DETECTION_USING_SNOW_LABS_PIPELINES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Class for Sentiment Analysis for News Articles**

## Colab Setup

In [11]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

     |████████████████████████████████| 212.4 MB 71 kB/s 
     |████████████████████████████████| 140 kB 41.8 MB/s 
     |████████████████████████████████| 198 kB 58.4 MB/s 
     |████████████████████████████████| 95 kB 2.6 MB/s 
     |████████████████████████████████| 66 kB 4.5 MB/s 


In [12]:
import sparknlp
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from tabulate import tabulate
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from sparknlp_display import NerVisualizer

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.4.0
Apache Spark version:  3.1.2


## Define a News Article

In [13]:
article = [ # two strings - headline & article body
"""Google sued in US over 'deceptive' location tracking""", # headline
"""Google is being sued in the US over accusations it deceived people about how to control location tracking.

The legal action refers to a widely reported 2018 revelation turning off one location-tracking setting in its apps was insufficient to fully disable the feature.

It accuses Google of using so-called dark patterns, marketing techniques that deliberately confuse.

Google said the claims were inaccurate and outdated.

'Unfair practices'
The legal action was filed in the District of Columbia. Similar ones were also filed in Texas, Indiana and Washington state.

It refers to an Associated Press revelation turning off Location History when using Google Maps or Search was insufficient - as a separate setting, Web and App Activity, continued to log location and other personal data.

The study, with researchers at Princeton University, found up to two billion Android and Apple devices could be affected.

"Google has relied on, and continues to rely on, deceptive and unfair practices that make it difficult for users to decline location tracking or to evaluate the data collection and processing to which they are purportedly consenting," the legal action alleges.

'Robust controls'
Google told BBC News the case was based "on inaccurate claims and outdated assertions about our settings".

A representative added: "We have always built privacy features into our products and provided robust controls for location data.

"We will vigorously defend ourselves and set the record straight."

Visual misdirection
The legal action claims Google's policies contained other "misleading, ambiguous and incomplete descriptions... but guarantee that consumers will not understand when their location is collected and retained by Google or for what purposes".

It refers to dark patterns, design choices that alter users' decision-making for the designer's benefit - such as, complicated navigation menus, visual misdirection, confusing wording and repeated nudging towards a particular outcome.

Data regulators are increasingly focusing on these practices.

Google faces a raft of other legal actions in the US, including:

In May 2020, Arizona filed a legal action over the same issue
In December 2020, multiple US states sued over the price and process of advertising auctions
In October 2020, the US Justice Department alleged Google had a monopoly over search and search advertising"""]



## Define the Brand Identification Class

In [14]:
class BrandIdentification:
    def __init__(self, MODEL_NAME):
        self.MODEL_NAME = MODEL_NAME

        # Define Spark NLP pipeline 
        documentAssembler = DocumentAssembler() \
            .setInputCol('text') \
            .setOutputCol('document')

        tokenizer = Tokenizer() \
            .setInputCols(['document']) \
            .setOutputCol('token')

        # ner_dl and onto_100 model are trained with glove_100d, so the embeddings in the pipeline should match
        if (self.MODEL_NAME == "ner_dl") or (self.MODEL_NAME == "onto_100"):
            embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
                .setInputCols(["document", 'token']) \
                .setOutputCol("embeddings")

        # Bert model uses Bert embeddings
        elif self.MODEL_NAME == "ner_dl_bert":
            embeddings = BertEmbeddings.pretrained(name='bert_base_cased', lang='en') \
                .setInputCols(['document', 'token']) \
                .setOutputCol('embeddings')

        ner_model = NerDLModel.pretrained(MODEL_NAME, 'en') \
            .setInputCols(['document', 'token', 'embeddings']) \
            .setOutputCol('ner')

        ner_converter = NerConverter() \
            .setInputCols(['document', 'token', 'ner']) \
            .setOutputCol('ner_chunk')

        nlp_pipeline = Pipeline(stages=[
            documentAssembler, 
            tokenizer,
            embeddings,
            ner_model,
            ner_converter
        ])
        
        # Create the pipeline model
        empty_df = spark.createDataFrame([['']]).toDF('text')
        self.pipeline_model = nlp_pipeline.fit(empty_df)


    def create_ranked_result_df(self, text):
        # Run the pipeline for the text
        text_df = spark.createDataFrame(pd.DataFrame({'text': text}, index = [0]))
        result = self.pipeline_model.transform(text_df)
        
        # Tabulate results
        df = result.select(F.explode(F.arrays_zip('document.result', 'ner_chunk.result',"ner_chunk.metadata")).alias("cols")).select(\
        F.expr("cols['1']").alias("chunk"),
        F.expr("cols['2'].entity").alias('result'))
        
        # Rank the identified ORGs by frequencies
        ranked_df = df.filter(df.result == 'ORG').groupBy(df.chunk).count().orderBy('count', ascending=False)

        return ranked_df


    def predict_by_headline(self, headline):
        ranked_df_hl = self.create_ranked_result_df(headline)
        ranked_df_hl.show(100, truncate=False)

        # If only one ORG appears in headline, return it 
        if ranked_df_hl.count() == 1:
            return ranked_df_hl.first()[0] 
        else: # If no ORG appears, or multiple ORGs all appear once, return no brand
            return None


    def predict(self, body):
        ranked_df = self.create_ranked_result_df(body)
        ranked_df.show(100, truncate=False)

        # Return the ORG with highest freq (at least greater than 2)
        if ranked_df.first()[1] > 2: 
            return ranked_df.first()[0] 
        else:
            return None
        # TO DO: break even - Wikidata#


    # def visualise(self, ranked_df, result):
        # Visualise ORG names in text
        # NerVisualizer().display(
            # result = result.collect()[0],
            # label_col = 'ner_chunk',
            # document_col = 'document',
            # labels=['ORG']
        #)


## Define the Senitment Identification Class

In [77]:
class SentimentIdentification:

    def __init__(self, MODEL_NAME):
        """Creates a class for sentiment identication using specified model.

        Args:
          MODEL_NAME: Name of the Spark NLP pretrained pipeline.
        """

        # Create the pipeline instance
        self.MODEL_NAME = MODEL_NAME

        if self.MODEL_NAME == "custom_pipeline": # https://nlp.johnsnowlabs.com/2021/11/03/bert_sequence_classifier_finbert_en.html
          document_assembler = DocumentAssembler() \
              .setInputCol('text') \
              .setOutputCol('document')

          tokenizer = Tokenizer() \
              .setInputCols(['document']) \
              .setOutputCol('token')

          sequenceClassifier = BertForSequenceClassification \
                .pretrained('bert_sequence_classifier_finbert', 'en') \
                .setInputCols(['token', 'document']) \
                .setOutputCol('class') \
                .setCaseSensitive(True) \
                .setMaxSentenceLength(512)

          pipeline = Pipeline(stages=[
              document_assembler,
              tokenizer,
              sequenceClassifier
          ])

          self.pipeline_model = LightPipeline(pipeline.fit(spark.createDataFrame([['']]).toDF("text")))

        else:
          self.pipeline_model = PretrainedPipeline(self.MODEL_NAME, lang = 'en')



    def predict(self, text):
        """Predicts sentiment of the input string..

        Args:
          text: String to classify.
        """
        self.text = text

        # Annotate input text using pretrained model
        annotations =  self.pipeline_model.annotate(self.text)

        # Depending on the chosen pipeline the outputs will be slightly different
        if self.MODEL_NAME == "analyze_sentimentdl_glove_imdb":
          # print(f"{annotations['sentiment']} {annotations['document']}")
          return annotations["sentiment"][0]

        else:
          # print(f"{annotations['class']} {annotations['document']}")
          return annotations['class'][0] # Return the sentiment string

## Identify Brand in news article


In [16]:
MODEL_NAME = "ner_dl_bert" # MODEL_NAME = "onto_100"

brand_identifier = BrandIdentification(MODEL_NAME)
headline, body = article

headline_brand = brand_identifier.predict_by_headline(headline)
print(headline)
print(headline_brand)

# Only use article body if no brand identified in the headline
if headline_brand == None:
    brand = brand_identifier.predict(body)
    print(brand)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]
ner_dl_bert download started this may take some time.
Approximate size to download 15.4 MB
[OK!]
+------+-----+
|chunk |count|
+------+-----+
|Google|1    |
+------+-----+

Google sued in US over 'deceptive' location tracking
Google


## Classify article using chosen pipeline

In [62]:
# identifier = SentimentIdentification(MODEL_NAME =  "analyze_sentimentdl_glove_imdb")
# identifier = SentimentIdentification(MODEL_NAME =  "classifierdl_bertwiki_finance_sentiment_pipeline")
identifier = SentimentIdentification(MODEL_NAME = "custom_pipeline") # Uses https://nlp.johnsnowlabs.com/2021/11/03/bert_sequence_classifier_finbert_en.html

# Predict by headline
headline = article[0]
identifier.predict(headline)

# Predict by body
body = article[1]
identifier.predict(body)


bert_sequence_classifier_finbert download started this may take some time.
Approximate size to download 390.9 MB
[OK!]


'negative'

## Test the accuracy of sentiment using the Financial News Headline Dataset

In [18]:
# Data downloaded from here: https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news/version/5
# Upload data from local machine
# from google.colab import files
# uploaded = files.upload()

### Alternatively download directly from kaggle using api keys

In [19]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news?select=all-data.csv")

# Input the following username and key when prompted:
# username: maxtimm
# Key: e9e0955b8d40b7d939e21febfbea8d15

ERROR: Operation cancelled by user
Skipping, found downloaded files in "./sentiment-analysis-for-financial-news" (use force=True to force download)


## Classify Kaggle Financial News Dataset

In [78]:
import io
# Store data in a Pandas Dataframe
df = pd.read_csv("./sentiment-analysis-for-financial-news/all-data.csv", encoding='latin-1')
df.columns = ['True Sentiment', 'Headline']

# shuffle the DataFrame rows
df = df.sample(frac = 1)

# Make dataset smaller for faster runtime
num_sentences = 1000
total_num_sentences = df.shape[0]
df.drop(df.index[num_sentences:total_num_sentences], inplace=True)

# Create the identifier object
identifier = SentimentIdentification(MODEL_NAME = "custom_pipeline") # 90.2% accuracy on 500 sentences 89.8% on 1000 sentences
# identifier = SentimentIdentification(MODEL_NAME =  "classifierdl_bertwiki_finance_sentiment_pipeline") # Alternative pretrained pipeline 90.0% accuracy on 500 sentences

preds = []
target = []
ignored_idxs = []
sentiment_to_ignore = "" # e.g. neutral

# Collect predicted sentiment for each headline - take three minutes to run
for idx, hl in enumerate(df['Headline']):

    # Only append the sentiment if it is not the sentiment to ignore (e.g. neutral)
    target_sentiment = df["True Sentiment"][df.index[idx]]

    if target_sentiment != sentiment_to_ignore:
      preds.append(identifier.predict(hl))
    else:
      ignored_idxs.append(idx)

    # Print progress
    if idx % 25 == 0:
      print(f"Classification {100*idx/num_sentences}% done.")

# Remove all ignored entries from dataset
df.drop(df.index[ignored_idxs], inplace=True)

df['Predicted Sentiment'] = preds

# Modify predicted labels to match with true labels
df = df.replace({'Predicted Sentiment': {'pos' : 'positive', 'neg' : 'negative'}})
df

bert_sequence_classifier_finbert download started this may take some time.
Approximate size to download 390.9 MB
[OK!]
Classification 0.0% done.
Classification 2.5% done.
Classification 5.0% done.
Classification 7.5% done.
Classification 10.0% done.
Classification 12.5% done.
Classification 15.0% done.
Classification 17.5% done.
Classification 20.0% done.
Classification 22.5% done.
Classification 25.0% done.
Classification 27.5% done.
Classification 30.0% done.
Classification 32.5% done.
Classification 35.0% done.
Classification 37.5% done.
Classification 40.0% done.
Classification 42.5% done.
Classification 45.0% done.
Classification 47.5% done.
Classification 50.0% done.
Classification 52.5% done.
Classification 55.0% done.
Classification 57.5% done.
Classification 60.0% done.
Classification 62.5% done.
Classification 65.0% done.
Classification 67.5% done.
Classification 70.0% done.
Classification 72.5% done.
Classification 75.0% done.
Classification 77.5% done.
Classification 80.0% 

,True Sentiment,Headline,Predicted Sentiment
3296,neutral,The new units should become one of the largest...,neutral
3939,neutral,The value of the three-year contract is estima...,neutral
2515,neutral,"Nastola , axe 14 jobs 16 February 2010 - Finni...",neutral
1785,positive,London 's leading shares today jumped almost 1...,positive
39,positive,"In January-September 2010 , Fiskars ' net prof...",positive
...,...,...,...
2468,positive,Growth is expected to continue in 2008 .,positive
4702,negative,Operating profit fell to EUR 6.2 mn from EUR 8...,negative
1910,neutral,"Sami Sepp+Ænen , CEO for Elisa Eesti , says th...",neutral
3017,neutral,( The acquisition sum has not been disclosed . ),neutral


In [79]:
from sklearn.metrics import classification_report

y_true = df['True Sentiment'].to_numpy()
y_pred = df['Predicted Sentiment'].to_numpy()

print(f"The accuracy is {100* sum(y_true==y_pred)/len(y_true)}%. \n")

target_names = ['positive', 'neutral', 'negative']

# Compute classification metrics - poor accuracy
print(classification_report(y_true, y_pred, target_names=target_names))

The accuracy is 89.8%. 

              precision    recall  f1-score   support

    positive       0.83      0.96      0.89       123
     neutral       0.95      0.88      0.91       587
    negative       0.84      0.91      0.87       290

    accuracy                           0.90      1000
   macro avg       0.87      0.92      0.89      1000
weighted avg       0.90      0.90      0.90      1000

